In [3]:
import os
from dotenv import load_dotenv
import requests
from huggingface_hub import InferenceClient

# Load environment variables from .env file
load_dotenv()

# Get token from .env file
hf_token = os.getenv('HUGGING_FACE_TOKEN')

if not hf_token:
    raise ValueError("HUGGING_FACE_TOKEN not found in .env file")

print("Token loaded successfully!")

Token loaded successfully!


In [ ]:

client = InferenceClient(api_key=hf_token)


In [ ]:
class StoryGenerator:
    def __init__(self, client,role="You are a story writer",max_tokens=3000,editor=None 
                 ,story_size="small", model="Qwen/Qwen2.5-Coder-32B-Instruct"):
        self.client = client
        self.toc = None
        self.chapter_titles = None
        self.story = None
        self.story_size = story_size
        self.model = model
        self.story_details = None
        self.num_chapters = 5
        self.system_role = role
        self.regenerate_chapters = False
        #add dynamic token handeling
        self.max_tokens =max_tokens

        #ToDo: Implement BookEditor first
        self.editor = editor

    def write_story(self):
      with open('generated_story.txt', 'w') as file:
          file.write(self.story)

      print("\nStory saved to 'generated_story.txt'")

    def determine_book_size(self,user_input="small"):
        
        if user_input == "small":
            self.story_size = "small"
        elif user_input == "medium":
            self.story_size = "medium"
        elif user_input == "large":
            self.story_size = "large"
        else:
            print("Invalid input. Defaulting to small.")
            self.story_size = "small"

    
    def build_all_chapters(self, chapter_titles):
      # Generate the content for each chapter
      self.story = self.toc + "\n\n"
      previous_summary = "Introduction to the story."  # Initial summary for the Prologue

      for i, chapter_name in enumerate(chapter_titles):
          previous_chapter = chapter_titles[i - 1] if i > 0 else "N/A"
          next_chapter = chapter_titles[i + 1] if i < len(chapter_titles) - 1 else "N/A"

          print(f"\nGenerating content for {chapter_name}...")
          until_end = len(chapter_titles) - i -1
          # Generate the content for the current chapter
          chapter_content, chapter_summary = self.generate_chapter(model=self.model,
             client=self.client, chapter_name=chapter_name, story_details=self.story_details, 
             previous_chapter=previous_chapter, next_chapter=next_chapter, previous_summary=previous_summary, 
             until_end= until_end
          )


          # Append the chapter content and summary to the story
          self.story += f"Chapter {i + 1}: {chapter_name}\n\n{chapter_content}\n\n"

          # Update the previous summary for the next iteration
          previous_summary = chapter_summary

      
    def get_completions(self,messages, client, model="Qwen/Qwen2.5-Coder-32B-Instruct",buffer=100, max_tokens=300):
      new_message = {"role": "assistant", "content": "Make sure your responses are complete and there are no cutoffs."}
      messages.append(new_message)
      completion = client.chat.completions.create(
                        model=self.model,
                        messages=messages,
                        max_tokens=max_tokens+buffer
                    )
      return completion

    def determine_dynamic_story_size(self, story_details):
        word_count = len(story_details.split())

        if word_count < 50:
            return "small"
        elif word_count < 200:
            return "medium"
        else:
            return "large"

    def build_story(self):
      user_input = input("Would you like the model to build a story for you.(yes/no)\n").strip()
      regen_input = input("Would you like to assist the model in improving chapters through chapter regeneration.(yes/no)\n").strip()
      if regen_input.lower() == "yes":
        print("Chapter regeneration is enabled.")
        self.regenerate_chapters = True
      else:
        print("Chapter regeneration is disabled")
        self.regenerate_chapters = False

      story_sizes = ["small","medium","large"]

      if user_input.lower() == "yes":
        

        # messages = [
        #     {"role": "system", "content": self.system_role},
        #     {"role": "user", "content": "How large is your story reply only with small, medium, large"},
        # ]
        # model_input = self.get_completions(messages, self.client, model="Qwen/Qwen2.5-Coder-32B-Instruct",buffer=100, max_tokens=100)
        # model_input = model_input.choices[0].message["content"].strip()
        # print(f'Story size set to: {model_input}')
        # self.determine_book_size(user_input=model_input)

        #Mabey should make story size use a randomizer instead of calling model
        idx = random.randint(0, len(story_sizes) - 1)
        story_size = story_sizes[idx]
        print(f'Story size set to: {story_size}')
        self.determine_book_size(user_input=story_size)

        if self.story_size == "small":
            max_tokens = 200
            buffer = 500
        elif self.story_size == "medium":
            max_tokens = 300
            buffer = 600
        elif self.story_size == "large":
            max_tokens = 400
            buffer = 700
        else:
          max_tokens = 212
          buffer = 500

        messages = [
            {"role": "system", "content": self.system_role},
            {"role": "user", "content": f"Describe your {self.story_size} sized story within {max_tokens} tokens. Be sure to highlight core story details and make sure your response only includes story details:"},
        ]
        model_input = self.get_completions(messages, self.client, model=self.model,buffer=buffer, max_tokens=max_tokens)
        model_input = model_input.choices[0].message["content"].strip()
        print(f'Generated Story Description: {model_input}')
        self.parse_story_details(self.client,user_input=model_input, model=self.model)
        

        messages = [
            {"role": "system", "content": self.system_role},
            {"role": "user", "content": f"How many chapters should the {self.story_size} sized story have with story details {model_input}? either choose random or choose a number between {2} and {10}: Make sure your response only includes random or a small number "},
        ]
        model_input = self.get_completions(messages, self.client, model=self.model,buffer=5, max_tokens=30)
        model_input = model_input.choices[0].message["content"].strip()
        
        self.determine_amount_of_chapters(chapter_input=model_input)
      else:
        # Initial story size determination
        user_input = input("How large is your story: small, medium, large?\n").strip()
        self.determine_book_size(user_input=user_input)

        # Story details input
        story_details = input("Describe your story. Be sure to highlight core story details:\n").strip()

        # Dynamically re-determine the size
        # new_size = self.determine_dynamic_story_size(story_details)
        # current_size = user_input

        # if new_size != current_size:
        #     print(f"The story size appears to be better suited as '{new_size}'. Do you accept this change? (yes/no)")
        #     change_size = input().strip().lower()

        #     if change_size == "yes":
        #         print(f"Story size has been updated to '{new_size}'.")
        #         self.determine_book_size(user_input=new_size)
        #     else:
        #         print(f"Keeping the story size as '{current_size}'.")

        # Proceed to parse story details
        self.parse_story_details(self.client, user_input=story_details, model=self.model)
        
        user_input = input("How many chapters should the story have? (Enter a number or type 'random'): ").strip()
        self.determine_amount_of_chapters(chapter_input=user_input)

      

      self.generate_toc(self.num_chapters)

      self.build_all_chapters(chapter_titles=self.chapter_titles)

      # print(self.story)
      self.write_story()
      return self.story

    def repair_truncated_json(self, raw_response):
      """
      Attempts to repair truncated JSON by appending missing brackets/braces.
      """
      try:
          return json.loads(raw_response)  # First try parsing as-is
      except json.JSONDecodeError:
          # If parsing fails, attempt to repair
          repaired_response = raw_response.rstrip(",}")  # Remove trailing commas or brackets
          stack = []  # Track opening braces and brackets
          for char in repaired_response:
              if char in "{[":
                  stack.append(char)
              elif char in "}]":
                  if stack and ((char == "}" and stack[-1] == "{") or (char == "]" and stack[-1] == "[")):
                      stack.pop()

          # Close remaining unbalanced braces/brackets
          while stack:
              last = stack.pop()
              repaired_response += "}" if last == "{" else "]"

          try:
              return json.loads(repaired_response)  # Try parsing repaired response
          except json.JSONDecodeError:
              print("Error: Unable to repair the JSON.")
              return None

    def update_story_details(self,details, story_summary):
        print("Updating story details")
        messages = [
                {"role": "system", "content": "You are a story writer that needs to update the current story details structured in valid JSON format."},
                {"role": "user", "content": f"Current details: {details}\n\nSummary of chapter:{story_summary}\n\nTry not to exceed {1000} tokens \n\nRespond with updated details in valid JSON format. Do not change current story details unless needed. Make sure you update/include details that are important to the story."}
            ]
        try:
            # completion = client.chat.completions.create(
            #     model=model,
            #     messages=messages,
            #     max_tokens=max_tokens+100
            # )
            completion = self.get_completions(messages=messages, model=self.model,client=self.client,max_tokens=2000,buffer=500)
            raw_response = completion.choices[0].message["content"].strip()
            self.story_details = self.repair_truncated_json(raw_response)
            if not self.story_details:
                print("Error: Could not parse or repair the JSON. Raw response:")
                print(raw_response)
                return None

            print("\nNew story Details:")
            print(json.dumps(self.story_details, indent=4))
            return self.story_details
        except Exception as e:
            print(f"Error during model interaction: {e}")
            return None

    def parse_story_details(self, client, details = None, model="Qwen/Qwen2.5-Coder-32B-Instruct",user_input="Suprise me with a story"):
        """
        Parses story details and ensures the output is complete and valid JSON.
        """
        if details == None:
          story_prompt = (
              "Describe your story. Include details about the plot, setting, timeline, main characters, "
              "and any other key elements. If some details are missing, the system will infer them for you."
          )
          
          
          if self.story_size == "small":
            max_tokens = 312
            buffer = 300
          elif self.story_size == "medium":
            max_tokens = 412
            buffer = 400
          elif self.story_size == "large":
            max_tokens = 512
            buffer = 500
          else:
            max_tokens = 312
            buffer =300

          messages = [
              {"role": "system", "content": "You are an assistant that creates structured story details in valid JSON format."},
              {"role": "user", "content": f"{story_prompt}\n\n{user_input}\n\nRespond with details in valid JSON format. Make sure that the response is less than {max_tokens} tokens."}
          ]
        else:
          if self.story_size == "small":
            max_tokens = 400
            buffer = 300
          elif self.story_size == "medium":
            max_tokens = 500
            buffer = 400
          elif self.story_size == "large":
            max_tokens = 600
            buffer = 500
          else:
            max_tokens = 212
            buffer = 200
          messages = [
              {"role": "system", "content": "You are an assistant that creates structured details in valid and complete JSON format."},
              {"role": "user", "content": f"Here are the details of the current story in json: {self.story_details}.Update the current story details with this factor to consider: {details}\n\nRespond with details in valid JSON format making sure to include a plot that summarizes all important details. Make sure that the response is less than {max_tokens} tokens."}
          ]

        try:
            if details == None:
              completion = client.chat.completions.create(
                  model=model,
                  messages=messages,
                  max_tokens=max_tokens+buffer
              )
            else:
              completion = client.chat.completions.create(
                  model=model,
                  messages=messages,
                  max_tokens=max_tokens+1000
              )
            # completion = self.get_completions(messages=messages, model=model,client=client,max_tokens=max_tokens,buffer=buffer)
            raw_response = completion.choices[0].message["content"].strip()
            self.story_details = self.repair_truncated_json(raw_response)
            if not self.story_details:
                print("Error: Could not parse or repair the JSON. Raw response:")
                print(raw_response)
                return None

            print("\nStory Details:")
            print(json.dumps(self.story_details, indent=4))
            return self.story_details
        except Exception as e:
            print(f"Error during model interaction: {e}")
            return None

    # Example usage ( need to replace `client` with your actual model client)
    # story_details = parse_story_details(client)

    def generate_chapter(
        self,
        client,
        chapter_name,
        story_details,
        previous_chapter,
        next_chapter,
        previous_summary,
        until_end,
        model="Qwen/Qwen2.5-Coder-32B-Instruct",
        chapter_details="N/a",  # Default to "N/a" if no specific chapter details are provided
        completion_criteria="Ensure this chapter has a clear beginning, middle, and end, with no loose ends."
    ):
        """
        Generate the content of a chapter given its context and ensure it is complete.
        The user will be able to view the chapter summary and decide whether to regenerate it or not.
        They can also specify any changes they'd like to apply to the story details.
        """
        if self.story_size == "small":
            max_tokens = 600
            buffer = 400
        elif self.story_size == "medium":
            max_tokens = 800
            buffer =600
        elif self.story_size == "large":
            max_tokens = 1000
            buffer = 800
        else:
          max_tokens = 412
          buffer = 200

        # Create the chapter generation prompt
        chapter_prompt = (
            f"Story details: {story_details}\n"
            f"Chapter details: {chapter_details}\n"
            f"Current chapter title: '{chapter_name}'.\n"
            f"Previous chapter title: '{previous_chapter}'.\n"
            f"Next chapter title: '{next_chapter}'.\n"
            f"Summary of the previous chapter: {previous_summary}\n"
            f"Chapters until the last chapter: {until_end}\n"
            f"{completion_criteria}\n"
            f"Write the content for this chapter. Be sure to include nothing but the content and not the title info. "
            f"Finish the chapter using no more than {max_tokens} tokens."
        )

        # Create the messages for the model
        messages = [
              {"role": "system", "content": "You are an assistant that creates story chapters using all the provided information."},
              {"role": "user", "content": f"{chapter_prompt}"}
          ]

        #make function get_completion
        try:
            # completion = client.chat.completions.create(
            #     model=model,
            #     messages=messages,
            #     max_tokens=max_tokens+200
            # )
            completion = self.get_completions(messages=messages, model=model,client=client,max_tokens=max_tokens,buffer=buffer)
        except Exception as e:
            print(f"Error during completion: {e}")
            return None, None
        
        # Get the generated chapter content
        chapter_content = completion.choices[0].message["content"].strip()

        # Summarize the chapter
        chapter_summary = self.summarize_chapter(chapter_content)

        # Present the summary and ask the user if they want to regenerate the chapter
        print(f"\nChapter {chapter_name} Summary:")
        print(chapter_summary)

        # regenerate_chapter(
        #     chapter_name,
        #     story_details,
        #     previous_chapter,
        #     next_chapter,
        #     previous_summary,
        #     until_end,
        #     chapter_details=chapter_details,
        #     completion_criteria=completion_criteria
        # )

        # Ask the user if they are satisfied with the chapter or if they want to regenerate
        if self.regenerate_chapters == True:
          regenerate_decision = input("\nDo you want to regenerate this chapter? (yes/no): ").strip().lower()

          if regenerate_decision == "yes":
              # Ask the user if they want to modify story details
              modify_decision = input("Would you like to modify the story details? (yes/no): ").strip().lower()

              additional_story_details = {}

              if modify_decision == "yes":
                  # Get additional details from the user to update story_details
                  additional_details = input("Please provide additional story details (or type 'none' if no modifications): ").strip()

                  if additional_details.lower() != 'none':
                      # Parse the new details using the same function
                      self.parse_story_details(client=client, details=additional_details)

                      # Ensure we merge the new details with the current story details
                      # if additional_story_details:
                      #     self.story_details.update(additional_story_details)
                      #     print(f"Story details updated with new information:\n {story_details}")
                      # else:
                      #     print("Error: Failed to parse new details. Proceeding with the existing story details.")

              # Extract plot from updated story details to pass as chapter_details
              new_chapter_details = self.story_details.get("plot", "N/a") if additional_story_details else chapter_details

              print(f"Regenerating chapter...plot:{new_chapter_details}")
              return self.generate_chapter(
                  client=self.client,
                  chapter_name=chapter_name,
                  #make  self.story_details if the the story_details are not too much
                  story_details=self.story_details,
                  previous_chapter=previous_chapter,
                  next_chapter=next_chapter,
                  previous_summary=previous_summary,
                  until_end=until_end,
                  completion_criteria=completion_criteria,
                  chapter_details=new_chapter_details,
                  model=model
                  
              )
        #may need to remove
        #self.update_story_details(self.story_details, chapter_summary)
        return chapter_content, chapter_summary



    def determine_amount_of_chapters(self,chapter_input=5):
      # Ask the user for chapter details
      if chapter_input.lower() == "random":
          self.num_chapters = random.randint(3, 15)  # Random number between 5 and 15
      else:
          try:
              self.num_chapters = int(chapter_input)
              print(f"Number of chapters in story: {self.num_chapters}")
          except ValueError:
              print("Invalid input. Defaulting to 5 chapters.")
              self.num_chapters = 5

    
    def generate_toc(self, num_chapters, model="Qwen/Qwen2.5-Coder-32B-Instruct"):
      self.toc = "Table of Contents\n\n"
      prev_title = "Prologue"  # Start with the Prologue
      self.toc += f"Chapter 1: {prev_title}\n"
      self.chapter_titles = [prev_title]  # Initialize with the first chapter's title

      for i in range(2, num_chapters + 1):
          # Join previous titles into a single string to include in the prompt
          previous_titles_str = ', '.join(self.chapter_titles)

          # Create the chapter prompt ensuring uniqueness
          chapter_prompt = (
              f"Here are the intitial details of the story {self.story_details} Previous chapter titles: '{previous_titles_str}'. "
              f"Suggest only the concise title for Chapter {i} based off the previous title {prev_title}. The title must be unique and not repeat any previous titles. "
              f"Make sure that your response is nothing other than than title of the chapter"
              f"The end of the story will be in {num_chapters-i} chapters make sure to consider this."
          )


          # Create the messages for the model
          messages = [
            {"role": "system", 
            "content": "You are a creative writer that creates good chapter titles."},
            {
              "role": "user",
              "content": f"{chapter_prompt}"
            }
          ]

          # Generate the title
          completion = client.chat.completions.create(
              model=self.model,
              messages=messages,
              max_tokens=15
          )

          next_title = completion.choices[0].message["content"].strip()
          # Ensure the title is unique by checking it against all previous titles

          # while next_title in self.chapter_titles:
          #     print(f"Duplicate title detected: {next_title}. Generating a new title...")

          #     # Regenerate the title if it's a duplicate
          #     completion = client.chat.completions.create(
          #         model=model,
          #         messages=messages,
          #         max_tokens=15
          #     )
          #     next_title = completion.choices[0].message["content"].strip()

          # Append the new title and update the table of contents
          self.chapter_titles.append(next_title)
          self.toc += f"Chapter {i}: {next_title}\n"

          # Update the previous title for the next iteration
          prev_title = next_title  # Update for the next iteration

    # Function to summarize a chapter
    def summarize_chapter(self,chapter_content, model="Qwen/Qwen2.5-Coder-32B-Instruct"):
        """Generate a brief summary of a chapter."""
        if self.story_size == "small":
            max_tokens = 512
            buffer = 300
        elif self.story_size == "medium":
            max_tokens = 612
            buffer = 400
        elif self.story_size == "large":
            max_tokens = 712
            buffer = 500
        else:
          max_tokens = 312
          buffer = 300
        
        summary_prompt = f"Summarize this chapter with the most important story facts in detailed sentences under {max_tokens} tokens: Be sure to mention all of the important story building details\n{chapter_content}"


        # Create the messages for the model
        messages = [
            {"role": "system", "content": "You are a creative writer that is trying to summarize chapters."},
            {
                "role": "user",
                "content": f"{summary_prompt}"
            }
        ]
        
        # Use the client to call the model and generate the summary
        # completion = self.client.chat.completions.create(
        #     model=model,  # Use the same model as before
        #     messages=messages,
        #     max_tokens=max_tokens +100  # You can adjust max_tokens as needed for the summary length
        # )
        completion = self.get_completions(messages=messages, model=model,client=client,max_tokens=max_tokens,buffer=buffer)

        # Get the generated summary
        summary = completion.choices[0].message["content"].strip()

        return summary




In [ ]:
gen = StoryGenerator(client)
gen.build_story()